# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [2]:
client.list_database_names()

['admin', 'config', 'ironhack', 'local']

In [3]:
db = client["ironhack"]

In [4]:
db.list_collection_names()

['companies', 'restaurants']

In [5]:
c = db.get_collection("companies")
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [6]:
# Your Code
filter_ = {"name":"Babelgum"}
projection = {"_id":0, "name":1} 
list(c.find(filter_, projection))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [7]:
# Your Code
filter_ = {"number_of_employees": {"$gt": 5000}}
projection = {"_id":0, "name":1, "number_of_employees":1}
list(c.find(filter_, projection).sort("number_of_employees", 1).limit(20))

[{'name': 'Nintendo', 'number_of_employees': 5080},
 {'name': 'Hexaware Technologies', 'number_of_employees': 5200},
 {'name': 'Facebook', 'number_of_employees': 5299},
 {'name': 'OpenText', 'number_of_employees': 5300},
 {'name': 'LSI', 'number_of_employees': 5400},
 {'name': 'CPM Braxis', 'number_of_employees': 5400},
 {'name': 'Microchip Technologies', 'number_of_employees': 5500},
 {'name': 'Mediaset', 'number_of_employees': 5729},
 {'name': 'Mindray Medical International', 'number_of_employees': 5763},
 {'name': 'Baidu', 'number_of_employees': 6000},
 {'name': 'Dentsu', 'number_of_employees': 6000},
 {'name': 'Tata Communications', 'number_of_employees': 6000},
 {'name': 'Atmel', 'number_of_employees': 6000},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'SRA International', 'number_of_employees': 6400},
 {'name': "Moody's", 'number_of_employees': 6800},
 {'name': 'ManTech', 'number_of_employees': 7000},
 {'name': 'Ad

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [ ]:
# Your Code
filter_1 = {"founded_year": {"$gte": 2000}},
filter_2 = {"founded_year": {"$lte": 2005}}
projection = {"_id":0, "name":1, "founded_year":1}
list(c.find({"$and": [filter_1, filter_2]}, projection))[:10]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [8]:
# Your Code
valuation = {"ipo.valuation_amount": {"$gt": 100000000}}
founded = {"founded_year": {"$lt":2010}}
projection = {"_id":0, "name":1, "ipo.valuation_amount":1}
list(c.find({"$and": [valuation, founded]}, projection))[:10]

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Brightcove', 'ipo': {'valuation_amount': 290000000}},
 {'name': 'KIT digital', 'ipo': {'valuation_amount': 235000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'OpenTable', 'ipo': {'valuation_amount': 1050000000}},
 {'name': 'ChannelAdvisor', 'ipo': {'valuation_amount': 287000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [9]:
# Your Code
employees = {"number_of_employees": {"$lt": 1000}}
founded = {"founded_year": {"$lt":2005}}
projection = {"_id":0, "name":1, "number_of_employees":1}
list(c.find({"$and": [employees, founded]}, projection).limit(10).sort("number_of_employees",-1))

[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Cornerstone OnDemand', 'number_of_employees': 881},
 {'name': 'Buongiorno', 'number_of_employees': 800},
 {'name': 'Cvent', 'number_of_employees': 800},
 {'name': 'ZoomInfo', 'number_of_employees': 800}]

### 6. All the companies that don't include the `partners` field.

In [10]:
# Your Code
list(c.find({"partners": {"$exists": False}}))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [11]:
# Your Code
query={"category_code":{"$type": 10}}
projection={"name":1,"_id":0}
list(c.find(query, projection).limit(10))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [12]:
# Your Code
employees = {"number_of_employees": {"$gt": 100}}
employees_ = {"number_of_employees": {"$lt":1000}}
projection = {"_id":0, "name":1, "number_of_employees":1}
list(c.find({"$and": [employees, employees_]}, projection).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [13]:
# Your Code
list(c.find({},{"_id":0, "name":1, "ipo.valuation_amount":1}).sort("ipo.valuation_amount",-1))[:10]

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [14]:
# Your Code
list(c.find({},{"_id":0, "name":1, "number_of_employees":1}).sort("number_of_employees",-1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [16]:
# Your Code
list(c.find({"founded_month":{"$gte":6}},{"_id":0, "name":1, "founded_month":1}).limit(1000))[:10]

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Postini', 'founded_month': 6},
 {'name': 'Geni', 'founded_month': 6},
 {'name': 'Fox Interactive Media', 'founded_month': 6},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [15]:
# Your Code
price = {"acquisition.price_amount": {"$gte": 10000}}
founded = {"founded_year": {"$lt": 2000}}
projection = {"_id":0, "name":1, "founded_year":1, "acquisition.price_amount":1}
list(c.find({"$and": [price,founded]}, projection))[:10]

[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'Cyworld',
  'founded_year': 1999,
  'acquisition': {'price_amount': 7140000}},
 {'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish',
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}},
 {'name': 'Neopets',
  'founded_year': 1999,
  'acquisition': {'price_amount': 160000000}},
 {'name': 'Sun Microsystems',
  'founded_year': 1982,
  'acquisition': {'price_amount': 7400000000}},
 {'name': 'Zappos',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1000000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [17]:
# Your Code
date = {"acquisition.acquired_year": {"$gt": 2010}}
projection = {"_id":0, "name":1, "acquisition.acquired_year":1}
list(c.find(date, projection).sort("acquisition.price_amount",-1))[:10]

[{'name': 'T-Mobile', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Goodrich Corporation', 'acquisition': {'acquired_year': 2011}},
 {'name': 'LSI', 'acquisition': {'acquired_year': 2013}},
 {'name': 'National Semiconductor', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Ariba', 'acquisition': {'acquired_year': 2012}},
 {'name': 'NetLogic Microsystems', 'acquisition': {'acquired_year': 2011}},
 {'name': 'SuccessFactors', 'acquisition': {'acquired_year': 2012}},
 {'name': 'Qualcomm Atheros', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Global Crossing', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Global Crossing', 'acquisition': {'acquired_year': 2011}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [18]:
# Your Code
list(c.find({},{"_id":0, "name":1, "founded_year":1}).sort("founded_year",-1))[:10]

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [19]:
# Your Code
founded_day = {"founded_day": {"$lte": 7, "$gt":0}}
projection = {"_id":0, "name":1, "founded_day":1, "acquisition.price_amount":1}
list(c.find(founded_day, projection).sort("acquisition.price_amount",-1).limit(10))

[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger',
  'founded_day': 1,
  'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems',
  'founded_day': 6,
  'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense',
  'founded_day': 1,
  'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee',
  'founded_day': 1,
  'acquisition': {'price_amount': 321000000}},
 {'name': 'Vitrue',
  'founded_day': 1,
  'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [20]:
# Your Code
employees = {"number_of_employees": {"$gt":4000}}
category = {"category_code":"web"}
projection = {"_id":0, "name":1, "number_of_employees":1}
list(c.find({"$and": [employees, category]}, projection).sort("number_of_employees"))[:10]

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [21]:
# Your Code
amount = {"acquisition.price_amount": {"$gt":10000000}}
currency = {"acquisition.price_currency_code":{"$eq":"EUR"}}
projection = {"_id":0, "name":1, "acquisition.price_amount":1, "acquisition.price_currency_code":1}
list(c.find({"$and": [amount, currency]}, projection)[:10])

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}},
 {'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'BioMed Central',
  'acquisition': {'price_amount': 43400000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [22]:
# Your Code
month = {"founded_month": {"$gte": 1,"$lte": 3}}
projection = {"_id":0, "name":1, "acquisition":1}
list(c.find(month, projection).limit(10))

[{'name': 'Facebook', 'acquisition': None},
 {'name': 'Twitter', 'acquisition': None},
 {'name': 'StumbleUpon',
  'acquisition': {'price_amount': 29000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2009/04/13/ebay-unacquires-stumbleupon/',
   'source_description': "StumbleUpon Beats Skype In Escaping EBay's Clutches",
   'acquired_year': 2009,
   'acquired_month': 4,
   'acquired_day': None,
   'acquiring_company': {'name': 'StumbleUpon', 'permalink': 'stumbleupon'}}},
 {'name': 'Gizmoz',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.gizmoz.com',
   'source_description': 'TechCrunch',
   'acquired_year': 2009,
   'acquired_month': 12,
   'acquired_day': 15,
   'acquiring_company': {'name': 'Daz 3d', 'permalink': 'daz-3d'}}},
 {'name': 'Helio',
  'acquisition': {'price_amount': 39000000,
   'price_currency_code': 'USD',
   'term_code': 'stock',
   'source_ur

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [23]:
# Your Code
founded_1 = {"founded_year": {"$gt": 2000}}
founded_2 = {"founded_year": {"$lt":2010}}
acq = {"acquisition.acquired_year": {"$gt":2011}}
projection = {"_id":0, "name":1, "founded_year":1, "acquisition.acquired_year": 1}
list(c.find({"$and": [founded_1, founded_2,acq]}, projection))[:10]

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'blogTV',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Revision3',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'iContact',
  'founded_year': 2003,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Mashery',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Dailymotion',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'KickApps',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Netvibes',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2012}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [24]:
# Your Code
dp = {"deadpooled_year": {"$gt":3}}
projection = {"_id":0, "name":1, "deadpooled_year":1}
list(c.find(dp, projection))[:10]

[{'name': 'Omnidrive', 'deadpooled_year': 2008},
 {'name': 'Babelgum', 'deadpooled_year': 2013},
 {'name': 'Sparter', 'deadpooled_year': 2008},
 {'name': 'Thoof', 'deadpooled_year': 2013},
 {'name': 'Mercora', 'deadpooled_year': 2008},
 {'name': 'Wesabe', 'deadpooled_year': 2010},
 {'name': 'Jangl SMS', 'deadpooled_year': 2008},
 {'name': 'Stickam', 'deadpooled_year': 2013},
 {'name': 'AllPeers', 'deadpooled_year': 2008},
 {'name': 'EQO', 'deadpooled_year': 2012}]